In [1]:
library(rliger)
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(SeuratDisk))
source("./MIRACLE/preprocess/utils.R")


Loading required package: Matrix



Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following object is masked from 'package:rliger':

    normalize


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: 'S4Vectors'


The following object is masked from 'package:future':

    values


The following objects are masked from 'package:Matrix':

 

In [2]:
config <- parseTOML("./MIRACLE/configs/data.toml")[['wnn_cellmask']]
subset_names <- basename(config$raw_data_dirs[1:8])
print(subset_names)
subset_ids <- sapply(seq_along(subset_names) - 1, toString)
input_dirs <- pj("./MIRACLE/result/wnn_cellmask/inputs/default/predict/", paste0("subset_", subset_ids))
pp_dir <- "./MIRACLE/data/processed/wnn_cellmask/"

[1] "p1_0" "p2_0" "p3_0" "p4_0" "p5_0" "p6_0" "p7_0" "p8_0"


In [26]:
z_list <- list()
cell_name_list <- list()
label_list <- list()
is_label <- T
subset_name_list <- list()
mask_list <- list()
S <- length(subset_names)
for (i in seq_along(subset_names)) {
    subset_name <- subset_names[i]
    z_dir    <- pj(input_dirs[i], "x", "rna")
    fnames <- dir(path = z_dir, pattern = ".csv$")
    fnames <- str_sort(fnames, decreasing = F)
    z_subset_list <- list()
    N <- length(fnames)
    for (n in seq_along(fnames)) {
        message(paste0("Loading Subset ", i, "/", S, ", File ", n, "/", N))
        z_subset_list[[n]] <- read.csv(file.path(z_dir, fnames[n]), header = F)
    }
    z_list[[subset_name]] <- bind_rows(z_subset_list)

    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "cell_names.csv"), header = T)[, 2]
    mask_list[[subset_name]] <- as.matrix(unname(read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "mask/rna.csv"), header = T)))
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}

rna <- t(data.matrix(bind_rows(z_list))) # pseudo rna counts
length(rna)
print(length(do.call("c", unname(cell_name_list))))
colnames(rna) <- do.call("c", unname(cell_name_list))
rownames(rna) <- paste0("rna-", seq_len(nrow(rna)))
obj <- CreateSeuratObject(counts = rna, assay = "rna")

Loading Subset 1/8, File 1/24



Loading Subset 1/8, File 2/24

Loading Subset 1/8, File 3/24

Loading Subset 1/8, File 4/24

Loading Subset 1/8, File 5/24

Loading Subset 1/8, File 6/24

Loading Subset 1/8, File 7/24

Loading Subset 1/8, File 8/24

Loading Subset 1/8, File 9/24

Loading Subset 1/8, File 10/24

Loading Subset 1/8, File 11/24

Loading Subset 1/8, File 12/24

Loading Subset 1/8, File 13/24

Loading Subset 1/8, File 14/24

Loading Subset 1/8, File 15/24

Loading Subset 1/8, File 16/24

Loading Subset 1/8, File 17/24

Loading Subset 1/8, File 18/24

Loading Subset 1/8, File 19/24

Loading Subset 1/8, File 20/24

Loading Subset 1/8, File 21/24

Loading Subset 1/8, File 22/24

Loading Subset 1/8, File 23/24

Loading Subset 1/8, File 24/24

Loading Subset 2/8, File 1/16

Loading Subset 2/8, File 2/16

Loading Subset 2/8, File 3/16

Loading Subset 2/8, File 4/16

Loading Subset 2/8, File 5/16

Loading Subset 2/8, File 6/16

Loading Subset 2/8, File 7/16

Loading Subset 2/8, File 8/16

Loading Subset 2/8, File

[1] 187162650

[1] 46213


In [27]:
mask <- mask_list[['p1_0']] + mask_list[['p2_0']] + mask_list[['p3_0']] + mask_list[['p4_0']] + mask_list[['p5_0']] + mask_list[['p6_0']] + mask_list[['p7_0']] + mask_list[['p8_0']] == 8

In [46]:
obj <- obj[mask[2:length(mask)], ]

In [47]:
z_list <- list()
cell_name_list <- list()
label_list <- list()
is_label <- T
subset_name_list <- list()
S <- length(subset_names)
for (i in seq_along(subset_names)) {
    subset_name <- subset_names[i]
    z_dir    <- pj(input_dirs[i], "x", "adt")
    fnames <- dir(path = z_dir, pattern = ".csv$")
    fnames <- str_sort(fnames, decreasing = F)
    z_subset_list <- list()
    N <- length(fnames)
    for (n in seq_along(fnames)) {
        message(paste0("Loading Subset ", i, "/", S, ", File ", n, "/", N))
        z_subset_list[[n]] <- read.csv(file.path(z_dir, fnames[n]), header = F)
    }
    z_list[[subset_name]] <- bind_rows(z_subset_list)

    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "cell_names.csv"), header = T)[, 2]
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}

rna <- t(data.matrix(bind_rows(z_list)))  # pseudo rna counts
length(rna)
print(length(do.call("c", unname(cell_name_list))))
colnames(rna) <- do.call("c", unname(cell_name_list))
rownames(rna) <- paste0("rna-", seq_len(nrow(rna)))
obj_adt <- CreateSeuratObject(counts = rna, assay = "adt")

Loading Subset 1/8, File 1/24

Loading Subset 1/8, File 2/24

Loading Subset 1/8, File 3/24

Loading Subset 1/8, File 4/24

Loading Subset 1/8, File 5/24

Loading Subset 1/8, File 6/24

Loading Subset 1/8, File 7/24

Loading Subset 1/8, File 8/24

Loading Subset 1/8, File 9/24

Loading Subset 1/8, File 10/24

Loading Subset 1/8, File 11/24

Loading Subset 1/8, File 12/24

Loading Subset 1/8, File 13/24

Loading Subset 1/8, File 14/24

Loading Subset 1/8, File 15/24

Loading Subset 1/8, File 16/24

Loading Subset 1/8, File 17/24

Loading Subset 1/8, File 18/24

Loading Subset 1/8, File 19/24

Loading Subset 1/8, File 20/24

Loading Subset 1/8, File 21/24

Loading Subset 1/8, File 22/24

Loading Subset 1/8, File 23/24

Loading Subset 1/8, File 24/24

Loading Subset 2/8, File 1/16

Loading Subset 2/8, File 2/16

Loading Subset 2/8, File 3/16

Loading Subset 2/8, File 4/16

Loading Subset 2/8, File 5/16

Loading Subset 2/8, File 6/16

Loading Subset 2/8, File 7/16

Loading Subset 2/8, File

[1] 10351712

[1] 46213


In [48]:
cat_data <- function(rna, adt){
    counts <- rbind(rna[['rna']]@counts, adt[['adt']]@counts)
    adata <- CreateSeuratObject(counts)
    return(adata)
}

In [49]:
adata <- cat_data(obj, obj_adt)
adata

Warning message:
"Non-unique features (rownames) present in the input matrix, making unique"


An object of class Seurat 
3675 features across 46213 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

In [50]:
scdata.list <- SplitObject(adata, split.by="orig.ident")
scdata.list

$B0
An object of class Seurat 
3675 features across 6142 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B1
An object of class Seurat 
3675 features across 3978 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B2
An object of class Seurat 
3675 features across 4103 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B3
An object of class Seurat 
3675 features across 5172 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B4
An object of class Seurat 
3675 features across 4205 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B5
An object of class Seurat 
3675 features across 5265 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B6
An object of class Seurat 
3675 features across 8770 samples within 1 assay 
Active assay: RNA (3675 features, 0 variable features)

$B7
An object of class Seurat 
3675 featu

In [51]:
batches=list()
i = 1
for(item in scdata.list){
    message(names(scdata.list[i]))
    batches[[names(scdata.list[i])]] = item@assays$RNA@counts
    i = i+1
}

B0

B1

B2

B3

B4

B5

B6

B7



In [52]:
adata_list = list()
adata_list[[names(scdata.list[1])]] = createLiger(batches[1], remove.missing=FALSE)
adata_list[[names(scdata.list[1])]] = rliger::normalize(adata_list[[names(scdata.list[1])]])
adata_list[[names(scdata.list[1])]]@var.genes = rownames(adata_list[[names(scdata.list[1])]]@raw.data$B0)
var.genes = rownames(adata_list[[1]]@raw.data$B0)
adata_list[[names(scdata.list[1])]] = scaleNotCenter(adata_list[[names(scdata.list[1])]])

for( i in c(2:8)){
adata_list[[names(scdata.list[i])]] = createLiger(batches[i], remove.missing=FALSE)
adata_list[[names(scdata.list[i])]] = rliger::normalize(adata_list[[names(scdata.list[i])]])
adata_list[[names(scdata.list[i])]]@var.genes = var.genes
adata_list[[names(scdata.list[i])]] = scaleNotCenter(adata_list[[names(scdata.list[i])]])
}
# }

In [10]:
for( i in c(1:8)){
    print(dim(adata_list[[names(scdata.list[i])]]@raw.data[[paste0('B', i-1)]]))
}

[1] 4274 6142
[1] 4274 3978
[1] 4274 4103
[1] 4274 5172
[1] 4274 4205
[1] 4274 5265
[1] 4274 8770
[1] 4274 8578


# Online

### train together but batch size set to 1

### train one batch by one batch

In [53]:
adata_list[[1]] = online_iNMF(adata_list[[1]], k = 32)
for(i in c(2:8)){
adata_list[[1]] = online_iNMF(adata_list[[1]], X_new=adata_list[i], k = 32,  miniBatch_size = 256)
}

Starting Online iNMF... 
  |======================================================================| 100%
Calculate metagene loadings... 
New dataset 1 already preprocessed. 
1 new datasets detected. 
Starting Online iNMF... 
  |======================================================================| 100%
Calculate metagene loadings... 
New dataset 1 already preprocessed. 
1 new datasets detected. 
Starting Online iNMF... 
  |======================================================================| 100%
Calculate metagene loadings... 
New dataset 1 already preprocessed. 
1 new datasets detected. 
Starting Online iNMF... 
  |======================================================================| 100%
Calculate metagene loadings... 
New dataset 1 already preprocessed. 
1 new datasets detected. 
Starting Online iNMF... 
  |======================================================================| 100%
Calculate metagene loadings... 
New dataset 1 already preprocessed. 
1 new datasets detected. 


In [54]:
adata_list[[1]] = quantile_norm(adata_list[[1]])

In [56]:
for(i in c(1:8)){
# dir.create(paste0('/opt/data/private/zjh/code/MIRACLE/result/wnn_cellmask/online_inmf_2024/default/predict/subset_',i-1,'/z/joint'), recursive = TRUE)
write.table(data.frame(adata_list[[1]]@H.norm)[adata[['orig.ident']]==paste0('B',i-1), ], paste0('/opt/data/private/zjh/code/MIRACLE/result/wnn_cellmask/online_inmf_2024/default/predict/subset_',i-1,'/z/joint/00.csv'), sep=',', row.names = F, col.names = F)
}

# offline

In [ ]:
adata = createLiger(batches, remove.missing=FALSE)
adata
adata = rliger::normalize(adata)
adata@var.genes = rownames(adata@raw.data$B0)
adata = scaleNotCenter(adata)

In [17]:
adata = online_iNMF(adata, k = 32, miniBatch_size = 5000, max.epochs = 5)

Starting Online iNMF... 
  |======================================================================| 100%
Calculate metagene loadings... 


In [18]:
adata = quantile_norm(adata)